In [91]:
import json

In [92]:
# importing related matches
with open('/Users/anesterov/reps/LODlit/bg/rm_bows_all.json','r') as jf:
    rm = json.load(jf)

In [93]:
# importing BoWs of search results
with open('/Users/anesterov/wd/jan31/wd_bows_en.json','r') as jf:
    wd_en = json.load(jf)
with open('/Users/anesterov/wd/jan31/wd_bows_nl.json','r') as jf:
    wd_nl = json.load(jf)
with open('/Users/anesterov/reps/LODlit/AAT/aat_bows_en.json','r') as jf:
    aat_en = json.load(jf)
with open('/Users/anesterov/reps/LODlit/AAT/aat_bows_nl.json','r') as jf:
    aat_nl = json.load(jf)
with open('/Users/anesterov/reps/LODlit/ODWN/odwn_bows.json','r') as jf:
    odwn = json.load(jf)
with open('/Users/anesterov/reps/LODlit/PWN/pwn31_bows.json','r') as jf:
    pwn = json.load(jf)

In [ ]:
# function for cosine similarity
def bows_cosine_similarity(resource:str,lang="en") -> str:
    '''
    Vectrorizes BoWs of search results and related matches
    Calculates cosine similarity between the two vectors
    term: str, query term
    resource: str, 'wikidata', 'aat', 'odwn', or 'pwn'
    lang: str, default is 'en'; could be 'nl' only for odwn and 'en' for pwn
    Saves a csv file with results
    Return a str "File is saved"
    '''
    # output: term, entity_id, resource, sim_rm (round)
    # read related matches and search results bows
    # importing related matches / change path
    with open('/Users/anesterov/reps/LODlit/bg/rm_bows_all.json','r') as jf:
        rm = json.load(jf)
        
    # checking resource and lang
    if resource == 'wikidata' and lang == 'en':
        
    #if resource == 'wikidata' and lang == 'nl':
        
    #if resource == 'aat' and lang == 'en':
        
    #if resource == 'aat' and lang == 'nl':
        
    #if resource == 'pwn':
    
    #if resource == 'odwn':
        
    related_matches = all_rm[lang][term]['wikidata'] + all_rm['en']['black']['aat'] + all_rm['en']['black']['pwn']
    wikidata = wd_en['black'][349]['Q53094']
    rm_set = {t for t in related_matches}
    wd_set = {t for t in wikidata}
    rvector = rm_set.union(wd_set)

In [28]:
lang = 'en'

In [37]:
rm_per_term = []
if rm[lang]['batavia'].get('aat'):
    rm_per_term.extend(rm[lang]['batavia']['aat'])
if rm[lang]['batavia'].get('wikidata'):
    rm_per_term.extend(rm[lang]['batavia']['wikidata'])
if rm[lang]['batavia'].get('pwn'):
    rm_per_term.extend(rm[lang]['batavia']['pwn'])

In [38]:
rm_per_term

['batavia',
 'capital',
 'city',
 'toponym',
 'capital',
 'dutch',
 'east',
 'indie',
 'city',
 'jakarta']

In [67]:
import pandas as pd

In [68]:
pd.DataFrame(columns=['a', 'b', 'c'])

,a,b,c


In [76]:
wd_df = pd.DataFrame(columns=['term', 'QID', 'wd_bow', 'cs'])

for term, hits in wd_en.items():
    
    if term in rm[lang].keys():
    
        # EN collecting related matches per term
        rm_per_term = []
        if rm[lang][term].get('aat'):
            rm_per_term.extend(rm[lang][term]['aat'])
        if rm[lang][term].get('wikidata'):
            rm_per_term.extend(rm[lang][term]['wikidata'])
        if rm[lang][term].get('pwn'):
            rm_per_term.extend(rm[lang][term]['pwn'])

        # remove target term from bow
        rm_per_term_no_target = [t for t in rm_per_term if term != t]
        # if there are search results
        if len(hits) > 0:
            for hit in hits:
                for i, bow in hit.items():
                    # remove target term from bow
                    wd_bow = [t for t in bow if t != term]
                    if len(wd_bow) > 0:
                        # calculate cs
                        cs = _vectorize_bows(rm_per_term_no_target,wd_bow)
                        wd_df.loc[len(wd_df)] = [term,i,wd_bow,cs]
                    # if there are no tokens except the target term, cs == None
                    else:
                        wd_df.loc[len(wd_df)] = [term,i,wd_bow,None]
                    
    # if there are no related matches, cs == None
    else:
        wd_df.loc[len(wd_df)] = [term,i,wd_bow,None]

In [78]:
wd_df.to_csv('cs_experiment.csv')

In [13]:
# subfunction: creating vectors

def _vectorize_bows(bow_1:list, bow_2:list) -> int:
    '''
    Vectorizing two BoWs
    Calculating cosine similarity between them
    bow_1 and bow_2: list, bag of words
    Returns int, rounded cosine similarity value
    '''
    # creating sets
    set_1 = {t for t in bow_1}
    set_2 = {t for t in bow_2}
    union_set = set_1.union(set_2)
    
    # vectorizing
    v1 = []
    v2 = []
    for t in union_set:
        if t in set_1:
            v1.append(1)
        else:
            v1.append(0)
        if t in set_2:
            v2.append(1)
        else:
            v2.append(0)
    
    # calculating cs 
    c = 0
    for i in range(len(union_set)):
        c += v1[i] * v2[i]

    cos_sim = c / float((sum(v1) * sum(v2)) ** 0.5)
    
    return round(cos_sim,3)

In [80]:
_vectorize_bows(related_matches['en']['madras']['wikidata'],['chennai', 'city', 'big', 'city', 'million', 'city', 'state', 'capital', 'business', 'cluster', 'megacity', 'capital', 'tamil', 'nadu', 'india'])

0.961

In [79]:
related_matches['en']['madras']['wikidata']

['chennai',
 'city',
 'big',
 'city',
 'million',
 'city',
 'state',
 'capital',
 'business',
 'cluster',
 'madras',
 'megacity',
 'capital',
 'tamil',
 'nadu',
 'india']

In [1]:
import spacy

In [2]:
import transformers

In [20]:
nlp = spacy.load("en_core_web_md")

In [138]:
nlp_l = spacy.load("en_core_web_lg", disable=["tagger", "attribute_ruler", "lemmatizer"])

In [21]:
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")

In [16]:
print(doc)

Apple shares rose on the news. Apple pie is delicious.


In [22]:
doc.vector

array([-1.18700646e-01, -1.41039336e+00, -2.14910340e+00,  8.83527994e-01,
        5.56492472e+00, -9.87121642e-01, -1.14053094e+00,  4.43576670e+00,
       -3.72914225e-01, -1.41204226e+00,  4.56952858e+00,  4.27760839e-01,
       -2.31775141e+00,  6.92854226e-01,  1.15528846e+00,  2.31034160e+00,
        2.57655549e+00, -6.75784111e-01,  1.13154316e+00, -2.44445491e+00,
        1.47608435e+00, -1.68519959e-01, -7.40572512e-01, -1.11758280e+00,
       -3.36654162e+00, -7.73094893e-01, -1.00135839e+00, -1.16324246e+00,
       -7.35800922e-01,  1.05197847e+00, -3.64668399e-01, -1.64117500e-01,
       -3.30009460e+00, -3.08980751e+00, -1.81026840e+00,  5.23460805e-01,
       -1.47361326e+00,  3.91166568e+00,  1.94414079e+00,  1.41275465e+00,
        9.01624143e-01,  2.62060571e+00, -1.04173517e-03, -9.47884858e-01,
        3.97454977e-01,  1.39844847e+00,  2.00074172e+00, -1.80079424e+00,
        8.34936619e-01,  9.27045763e-01, -7.79365063e-01,  1.24467003e+00,
       -1.04951508e-01, -

In [35]:
apples = nlp("chennai city big million state capital business cluster megacity tamil nadu india'")
oranges = nlp("Madras is a city")
apples_oranges = apples.similarity(oranges)

In [81]:
apple_1 = nlp("black people colour skin african american race")
apple_2 = nlp("black colour darkest grey paint")

In [36]:
apples_oranges

0.5233238157706054

In [82]:
sim = apple_1.similarity(apple_2)

In [83]:
sim

0.7018133052155686

In [179]:
apple_1 = nlp_l(rm_str)
apple_2 = nlp_l(wd_str)

In [180]:
apple_1.similarity(apple_2)

0.7892355337039458

In [187]:
type(apple_1)

spacy.tokens.doc.Doc

In [190]:
for token in apple_2:
    print(token.text,token.vector_norm)

life 68.47362
matter 45.858288
social 63.778107
movement 52.706676
blm 3.5632813
international 47.453915
human 63.08982
right 42.769985
movement 52.706676


In [191]:
for token in apple_1:
    print(token.text,token.vector_norm)

people 51.63681
race 66.79318
human 63.08982
categorization 39.347366
person 50.16031
color 57.97093
man 68.725006
woman 53.992878
person 50.16031
racialized 34.79334
classification 43.620155
people 51.63681
usually 43.34739
political 53.04342
skin 78.87063
color 57.97093
based 65.48822
category 37.327602
specific 46.39293
population 56.26217
mid 82.85134
dark 54.89067
brown 47.859325
complexion 34.19395


In [153]:
apple_1[2]

people

In [111]:
apple_3 = nlp_l("eye color, black eyes, black-eyed, black eye color, black eye colour,\
                black (eye color), black (eye colour), black (eye), black (eyes), black-eyed person,black-eyed human")

In [114]:
apple_4 = nlp_l("hole, astronomical object so massive, that anything falling into it,\
                including light, cannot escape its gravity, astronomical object type")

In [177]:
wd_str = ""
for token in wd_en["black"][151]["Q19600530"]:
    if token != "black":
        wd_str = wd_str + f"{token} "

In [178]:
wd_str

'life matter social movement blm international human right movement '

In [157]:
rm_str = ""
for t in rm["en"]["blacks"]["wikidata"]:
    if t != "blacks":
        rm_str = rm_str + f"{t} "

In [181]:
rm_str

'people race human categorization person color man woman person racialized classification people usually political skin color based category specific population mid dark brown complexion '

In [1]:
import scipy

In [2]:
from scipy import spatial

In [ ]:
1 - spatial.distance.cosine(dataSetI, dataSetII)